* Model based recsys
    * ALS / Matrix factorization 으로 추천 결과 및 성능 비교하기

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.options.display.float_format = '{:.2f}'.format
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, mean_squared_error

In [63]:
# 데이터셋 로드
data_path = './anime/'

anime = pd.read_csv(data_path+'anime.csv')
rating = pd.read_csv(data_path+'rating.csv').sample(n=2500000)

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
rating.shape

(2500000, 3)

In [5]:
rating.head()

,user_id,anime_id,rating
5359528,50752,507,-1
1023515,9746,1281,8
5986091,56120,26443,6
6926135,64559,3901,-1
2080738,20146,13469,6


#### preprocessing

In [64]:
print(rating.shape)
rating['rating'].describe() # rating - rating out of 10 this user has assigned (-1 if the user watched it but didn't assign a rating).

(2500000, 3)


count   2500000.00
mean          6.14
std           3.73
min          -1.00
25%           6.00
50%           7.00
75%           9.00
max          10.00
Name: rating, dtype: float64

In [65]:
rating['original_rating'] = rating['rating']
rating['rating'] = [1 if x>=6 else 0 for x in rating['rating']]

In [8]:
rating.loc[rating['user_id']==1,'rating'].describe() # 어떤 user는 시청한 대부분의 item의 rating 을 매기지 않음

count   52.00
mean     0.04
std      0.19
min      0.00
25%      0.00
50%      0.00
75%      0.00
max      1.00
Name: rating, dtype: float64

In [9]:
pd.merge(rating.loc[(rating['user_id']==1)&(rating['rating']>0)], anime, on ='anime_id', how='left')

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,15451,1,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",TV,12,7.87,266657
1,1,11757,1,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,25,7.83,893100


In [10]:
# 모든 item 의 rating 을 assign 하지 않은 경우? >> 3915명
# 이 경우에는 item feature 기반 contents based filtering 으로 추천
tmp =rating.groupby('user_id')['rating'].agg(['count','sum'])
tmp['tmp_val'] = tmp['count']+tmp['sum']  
tmp[tmp['tmp_val']==0]

,count,sum,tmp_val
user_id,,,


In [11]:
exclude_users = list(tmp[tmp['tmp_val']==0].index)
train_users_interaction = rating.loc[~rating['user_id'].isin(exclude_users)]
print(train_users_interaction.shape)

(2500000, 3)


In [12]:
# 위 interaction dataframe 내 -1 rating 은 item dataframe 의 rating 으로 대체

In [13]:
train_users_interaction.loc[train_users_interaction['rating']==-1,'rating'] = None

In [14]:
len([c.strip() for c in anime['genre'].str.get_dummies(sep=',').columns])

82

In [15]:
anime_features = anime[['anime_id','episodes','rating','members']].join(anime['genre'].str.get_dummies(sep=', '))
anime_features.columns

Index(['anime_id', 'episodes', 'rating', 'members', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen', 'Shounen Ai',
       'Slice of Life', 'Space', 'Sports', 'Super Power', 'Supernatural',
       'Thriller', 'Vampire', 'Yaoi', 'Yuri'],
      dtype='object')

In [16]:
anime_features = pd.concat([anime_features, anime['type'].str.get_dummies()],axis=1)
anime_features.shape

(12294, 53)

In [17]:
genre_type_cols = list(set([c.strip() for c in anime_features.columns if c not in ['anime_id','episodes','rating','members']]))
print(len(genre_type_cols))
print(genre_type_cols)

48
['Police', 'Space', 'Mecha', 'Vampire', 'Slice of Life', 'Drama', 'Adventure', 'Sports', 'Game', 'Dementia', 'Josei', 'Shounen', 'Yuri', 'ONA', 'Shoujo', 'Super Power', 'Military', 'Hentai', 'Sci-Fi', 'TV', 'Fantasy', 'Movie', 'School', 'Demons', 'Action', 'Comedy', 'Kids', 'Shoujo Ai', 'Parody', 'Cars', 'Yaoi', 'Ecchi', 'Samurai', 'Music', 'Supernatural', 'Seinen', 'Psychological', 'Martial Arts', 'Special', 'Harem', 'Mystery', 'Thriller', 'Romance', 'Shounen Ai', 'Magic', 'Horror', 'OVA', 'Historical']


In [18]:
anime_features = anime[['anime_id','episodes','rating','members']].join(anime['genre'].str.get_dummies(sep=', '))
anime_features = pd.concat([anime_features, anime['type'].str.get_dummies()],axis=1)
genre_type_cols = list(set([c.strip() for c in anime_features.columns if c not in ['anime_id','episodes','rating','members']]))
print(len(genre_type_cols))
print(genre_type_cols)

anime_features.head()

48
['Police', 'Space', 'Mecha', 'Vampire', 'Slice of Life', 'Drama', 'Adventure', 'Sports', 'Game', 'Dementia', 'Josei', 'Shounen', 'Yuri', 'ONA', 'Shoujo', 'Super Power', 'Military', 'Hentai', 'Sci-Fi', 'TV', 'Fantasy', 'Movie', 'School', 'Demons', 'Action', 'Comedy', 'Kids', 'Shoujo Ai', 'Parody', 'Cars', 'Yaoi', 'Ecchi', 'Samurai', 'Music', 'Supernatural', 'Seinen', 'Psychological', 'Martial Arts', 'Special', 'Harem', 'Mystery', 'Thriller', 'Romance', 'Shounen Ai', 'Magic', 'Horror', 'OVA', 'Historical']


,anime_id,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,32281,1,9.37,200630,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,5114,64,9.26,793665,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,28977,51,9.25,114262,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,9253,24,9.17,673572,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,9969,51,9.16,151266,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
total_df = pd.merge(train_users_interaction, anime_features, on='anime_id', how='left')
total_df.head()

,user_id,anime_id,rating_x,episodes,rating_y,members,Action,Adventure,Cars,Comedy,...,Thriller,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV
0,50752,507,0.00,2,7.12,22015.00,0.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
1,9746,1281,1.00,19,7.71,42033.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
2,56120,26443,1.00,10,6.44,59632.00,1.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
3,64559,3901,0.00,3,8.29,100412.00,1.00,0.00,0.00,1.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00
4,20146,13469,1.00,1,7.43,68823.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [20]:
total_df['rating'] = np.where(total_df['rating_x'].isnull(), total_df['rating_y'],total_df['rating_x'])
del total_df['rating_x']
del total_df['rating_y']
total_df.head()

,user_id,anime_id,episodes,members,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV,rating
0,50752,507,2,22015.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,9746,1281,19,42033.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
2,56120,26443,10,59632.00,1.00,1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
3,64559,3901,3,100412.00,1.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,20146,13469,1,68823.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00


In [21]:
total_df.groupby('user_id')['rating'].mean().rename('user_avg_rating').reset_index()

,user_id,user_avg_rating
0,1,0.04
1,3,0.87
2,4,0.00
3,5,0.33
4,6,0.00
...,...,...
69970,73512,0.67
69971,73513,1.00
69972,73514,1.00
69973,73515,0.87


In [22]:
total_df.head()

,user_id,anime_id,episodes,members,Action,Adventure,Cars,Comedy,Dementia,Demons,...,Vampire,Yaoi,Yuri,Movie,Music,ONA,OVA,Special,TV,rating
0,50752,507,2,22015.00,0.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,9746,1281,19,42033.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
2,56120,26443,10,59632.00,1.00,1.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00
3,64559,3901,3,100412.00,1.00,0.00,0.00,1.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,20146,13469,1,68823.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00


In [23]:
def get_feature_set(df):
    
    X = df.drop(columns=['rating'])
    y = df[['rating']]
    print('# of original features :::: {}'.format(X.shape[1]))
    # feature engineering

    X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=.25, random_state=42)

    # train set user based feature engineering

    # user baesd feature engineering
    user_feature_genre_rating = pd.concat([X_train,y_train],axis=1).reset_index(drop=True).melt(id_vars=['user_id', 'rating'], value_vars=genre_type_cols) \
    .query("value == 1") \
    .groupby(['user_id', 'variable'])['rating'].mean().unstack().fillna(0)

    user_feature_avg_rating = pd.concat([X_train,y_train],axis=1).groupby('user_id')['rating'].mean().rename('user_avg_rating').reset_index()
    user_feature_cnt_rating = pd.concat([X_train,y_train],axis=1).groupby('user_id')['rating'].count().rename('user_cnt_rating').reset_index()
    user_feature_std_rating = pd.concat([X_train,y_train],axis=1).groupby('user_id')['rating'].std().rename('user_std_rating').reset_index()

    # item based feature engineering
    item_feature_avg_rating = pd.concat([X_train,y_train],axis=1).groupby('anime_id')['rating'].mean().rename('item_avg_rating').reset_index()
    item_feature_cnt_rating = pd.concat([X_train,y_train],axis=1).groupby('anime_id')['rating'].count().rename('item_cnt_rating').reset_index()
    item_feature_std_rating = pd.concat([X_train,y_train],axis=1).groupby('anime_id')['rating'].std().rename('item_std_rating').reset_index()

    X_train_interaction = X_train[['user_id','anime_id']]
    X_train_concat = pd.merge(X_train_interaction,user_feature_genre_rating, on='user_id',how='left')
    X_train_concat = pd.merge(X_train_concat,user_feature_avg_rating, on='user_id',how='left')
    X_train_concat = pd.merge(X_train_concat,user_feature_cnt_rating, on='user_id',how='left')  
    X_train_concat = pd.merge(X_train_concat,user_feature_std_rating, on='user_id',how='left')
    X_train_concat = pd.merge(X_train_concat,item_feature_avg_rating, on='anime_id',how='left')
    X_train_concat = pd.merge(X_train_concat,item_feature_cnt_rating, on='anime_id',how='left')
    X_train_concat = pd.merge(X_train_concat,item_feature_std_rating, on='anime_id',how='left')

    
    X_test_interaction = X_test[['user_id','anime_id']]
    X_test_concat = pd.merge(X_test_interaction,user_feature_genre_rating, on='user_id',how='left')
    X_test_concat = pd.merge(X_test_concat,user_feature_avg_rating, on='user_id',how='left')
    X_test_concat = pd.merge(X_test_concat,user_feature_cnt_rating, on='user_id',how='left')  
    X_test_concat = pd.merge(X_test_concat,user_feature_std_rating, on='user_id',how='left')
    X_test_concat = pd.merge(X_test_concat,item_feature_avg_rating, on='anime_id',how='left')
    X_test_concat = pd.merge(X_test_concat,item_feature_cnt_rating, on='anime_id',how='left')
    X_test_concat = pd.merge(X_test_concat,item_feature_std_rating, on='anime_id',how='left')

    X_train_concat = X_train_concat.fillna(0.0)
    X_test_concat = X_test_concat.fillna(0.0)

    return X_train_concat, X_test_concat, y_train, y_test

In [24]:
X_train,X_test,y_train, y_test = get_feature_set(total_df)

# of original features :::: 53


In [25]:
X_train.to_pickle('./anime/X_train_anime.pkl')
X_test.to_pickle('./anime/X_test_anime.pkl')
y_train.to_pickle('./anime/y_train_anime.pkl')
y_test.to_pickle('./anime/y_test_anime.pkl')

In [5]:
X_train = pd.read_pickle('./anime/X_train_anime.pkl')
y_train = pd.read_pickle('./anime/y_train_anime.pkl')

In [25]:
X_train.head()

,user_id,anime_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,...,Thriller,Vampire,Yaoi,Yuri,user_avg_rating,user_cnt_rating,user_std_rating,item_avg_rating,item_cnt_rating,item_std_rating
0,20964,6547,0.80,1.00,0.00,0.67,0.00,1.00,0.67,1.00,...,1.00,0.50,0.00,0.00,0.62,13,0.51,0.85,6481,0.36
1,44460,4081,0.75,0.50,0.00,1.00,0.00,0.00,1.00,1.00,...,1.00,0.00,0.00,0.00,0.82,11,0.40,0.82,1445,0.38
2,72212,18671,0.03,0.00,0.00,0.00,0.00,0.00,0.29,0.00,...,0.00,0.00,0.00,0.00,0.09,74,0.29,0.77,1702,0.42
3,40343,11617,0.89,1.00,0.00,0.93,0.00,1.00,1.00,1.00,...,0.00,0.50,0.00,0.00,0.91,22,0.29,0.77,3502,0.42
4,39477,5521,0.86,0.84,1.00,0.87,0.00,0.90,1.00,0.94,...,1.00,1.00,0.00,0.00,0.88,241,0.33,0.75,359,0.43


In [42]:
X_test = pd.read_pickle('./anime/X_test_anime.pkl')
y_test = pd.read_pickle('./anime/y_test_anime.pkl')

In [38]:
X_test.head()

,user_id,anime_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,...,Thriller,Vampire,Yaoi,Yuri,user_avg_rating,user_cnt_rating,user_std_rating,item_avg_rating,item_cnt_rating,item_std_rating
0,65594,3455,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,13,0.00,0.74,2118,0.44
1,47291,8142,0.95,0.98,1.00,0.85,1.00,1.00,1.00,0.45,...,1.00,0.75,0.00,0.00,0.93,176,0.26,0.81,451,0.39
2,25773,10431,1.00,1.00,0.00,1.00,0.00,1.00,1.00,1.00,...,1.00,1.00,1.00,0.00,1.00,85,0.00,0.64,237,0.48
3,2695,31772,0.70,0.73,0.00,0.74,0.67,0.50,0.76,0.62,...,1.00,1.00,0.00,0.00,0.75,157,0.44,0.73,279,0.44
4,37659,7724,0.85,0.85,0.00,0.74,0.00,0.50,0.88,0.60,...,0.71,1.00,0.00,0.00,0.72,100,0.45,0.83,1990,0.38


In [26]:
y_train.shape, y_test.shape

((1875000, 1), (625000, 1))

In [27]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Layer
from tensorflow.keras.models import Model

In [28]:
class FactorizationMachineLayer(Layer):
    def __init__(self, **kwargs):
        super(FactorizationMachineLayer, self).__init__(**kwargs)

    def call(self, x):
        square_of_sum = tf.square(tf.reduce_sum(x, axis=1))
        sum_of_square = tf.reduce_sum(tf.square(x), axis=1)
        interaction = 0.5 * tf.reduce_sum(square_of_sum - sum_of_square, axis=1, keepdims=True)
        return interaction

def build_fm_with_embedding_model(n_users, n_items, n_numeric, embed_dim=10):
    # ▶️ 입력 정의
    user_input = Input(shape=(1,), name='user_id')
    item_input = Input(shape=(1,), name='anime_id')
    num_input = Input(shape=(n_numeric,), name='numeric_features')

    # ▶️ 임베딩
    user_embed = Embedding(input_dim=n_users, output_dim=embed_dim, name='user_embedding')(user_input)
    item_embed = Embedding(input_dim=n_items, output_dim=embed_dim, name='item_embedding')(item_input)

    user_vec = Flatten()(user_embed)
    item_vec = Flatten()(item_embed)

    # ▶️ FM 2차 interaction을 위한 벡터 결합
    fm_input = Concatenate(axis=1)([user_vec, item_vec, num_input])
    fm_expanded = tf.expand_dims(fm_input, axis=1)

    # ▶️ FM interaction 레이어
    interaction_output = FactorizationMachineLayer()(fm_expanded)

    # ▶️ 1차 선형항: Dense layer로 구성
    linear_part = Dense(1)(fm_input)

    # ▶️ 최종 output
    output = tf.keras.layers.Add()([linear_part, interaction_output])

    model = Model(inputs=[user_input, item_input, num_input], outputs=output)
    return model

In [29]:
X_train.shape, X_test.shape

((1875000, 56), (625000, 56))

In [30]:
# 입력 feature 수
n_users = X_train['user_id'].nunique()
n_items = X_train['anime_id'].nunique()
n_numeric = X_train.shape[1] - 2  # user_id, anime_id 제외한 feature 수

# 모델 생성
model = build_fm_with_embedding_model(n_users=n_users, n_items=n_items, n_numeric=n_numeric, embed_dim=10)
model.compile(optimizer='adam', loss='mse')
model.summary()

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2025-03-31 09:43:40.167550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-31 09:43:40.167994: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_id (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 anime_id (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 10)        686810      ['user_id[0][0]']                
                                                                                                  
 item_embedding (Embedding)     (None, 1, 10)        93970       ['anime_id[0][0]']               
                                                                                              

In [32]:
# 모델 입력
X = {
    'user_id': X_train['user_id'].values,
    'anime_id': X_train['anime_id'].values,
    'numeric_features': X_train.drop(columns=['user_id', 'anime_id']).values
}
y = y_train

model.fit(X, y, epochs=10, batch_size=512)

Epoch 1/10


2025-03-31 09:53:41.282819: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2025-03-31 09:53:41.586698: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3663/3663 [==============================] - 29s 7ms/step - loss: 4323.1831
Epoch 2/10
3663/3663 [==============================] - 25s 7ms/step - loss: 0.2382
Epoch 3/10
3663/3663 [==============================] - 25s 7ms/step - loss: 0.1041
Epoch 4/10
3663/3663 [==============================] - 25s 7ms/step - loss: 0.0920
Epoch 5/10
3663/3663 [==============================] - 25s 7ms/step - loss: 0.0924
Epoch 6/10
3663/3663 [==============================] - 25s 7ms/step - loss: 0.0903
Epoch 7/10
3663/3663 [==============================] - 25s 7ms/step - loss: 0.0899
Epoch 8/10
3663/3663 [==============================] - 27s 7ms/step - loss: 0.0885
Epoch 9/10
3663/3663 [==============================] - 27s 7ms/step - loss: 0.0887
Epoch 10/10
3663/3663 [==============================] - 27s 7ms/step - loss: 0.0883


In [43]:
X_test_dict = {
    'user_id': X_test['user_id'].values,
    'anime_id': X_test['anime_id'].values,
    'numeric_features': X_test.drop(columns=['user_id', 'anime_id']).values
}

In [51]:
y_pred = model.predict(X_test_dict)

58594/58594 [==============================] - 130s 2ms/step


In [76]:
def recommend_topk_items(model, user_id, item_ids, user_feature_df, item_feature_df, k=5):
    # user_id 단일값에 대한 feature 생성
    user_features = user_feature_df[user_feature_df['user_id'] == user_id].drop(columns=['user_id']).values
    if len(user_features) == 0:
        raise ValueError(f"user_id {user_id} not found in user_feature_df")
    
    # 모든 item_id에 대해 feature 조합 생성
    input_data = []
    for item_id in item_ids:
        item_features = item_feature_df[item_feature_df['anime_id'] == item_id].drop(columns=['anime_id']).values
        if len(item_features) == 0:
            continue
        # 단일 user + item 조합
        sample = {
            'user_id': np.array([user_id]),
            'anime_id': np.array([item_id]),
            'numeric_features': np.concatenate([user_features[0], item_features[0]])[np.newaxis, :]
        }
        input_data.append((item_id, sample))
    
    # 예측 수행
    scored = []
    for item_id, sample_input in input_data:
        pred_score = model.predict(sample_input, verbose=0)[0][0]
        scored.append((item_id, pred_score))
    
    # 상위 k개 반환
    top_k = sorted(scored, key=lambda x: x[1], reverse=True)[:k]
    return [(item,score) for item, score in top_k]

In [52]:
X_train.head()

,user_id,anime_id,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,...,Thriller,Vampire,Yaoi,Yuri,user_avg_rating,user_cnt_rating,user_std_rating,item_avg_rating,item_cnt_rating,item_std_rating
0,20964,6547,0.80,1.00,0.00,0.67,0.00,1.00,0.67,1.00,...,1.00,0.50,0.00,0.00,0.62,13,0.51,0.85,6481,0.36
1,44460,4081,0.75,0.50,0.00,1.00,0.00,0.00,1.00,1.00,...,1.00,0.00,0.00,0.00,0.82,11,0.40,0.82,1445,0.38
2,72212,18671,0.03,0.00,0.00,0.00,0.00,0.00,0.29,0.00,...,0.00,0.00,0.00,0.00,0.09,74,0.29,0.77,1702,0.42
3,40343,11617,0.89,1.00,0.00,0.93,0.00,1.00,1.00,1.00,...,0.00,0.50,0.00,0.00,0.91,22,0.29,0.77,3502,0.42
4,39477,5521,0.86,0.84,1.00,0.87,0.00,0.90,1.00,0.94,...,1.00,1.00,0.00,0.00,0.88,241,0.33,0.75,359,0.43


In [53]:
X_train.columns

Index(['user_id', 'anime_id', 'Action', 'Adventure', 'Cars', 'Comedy',
       'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy', 'Game', 'Harem',
       'Hentai', 'Historical', 'Horror', 'Josei', 'Kids', 'Magic',
       'Martial Arts', 'Mecha', 'Military', 'Movie', 'Music', 'Mystery', 'ONA',
       'OVA', 'Parody', 'Police', 'Psychological', 'Romance', 'Samurai',
       'School', 'Sci-Fi', 'Seinen', 'Shoujo', 'Shoujo Ai', 'Shounen',
       'Shounen Ai', 'Slice of Life', 'Space', 'Special', 'Sports',
       'Super Power', 'Supernatural', 'TV', 'Thriller', 'Vampire', 'Yaoi',
       'Yuri', 'user_avg_rating', 'user_cnt_rating', 'user_std_rating',
       'item_avg_rating', 'item_cnt_rating', 'item_std_rating'],
      dtype='object')

In [54]:
merge_df = pd.concat([X_train,X_test]).reset_index(drop=True)
user_feature_df = merge_df[['user_id','user_avg_rating', 'user_cnt_rating', 'user_std_rating']]
item_feature_df = merge_df[[c for c in merge_df.columns if c not in user_feature_df.columns]]

In [55]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [56]:
sample_user_id = 20964
watched_ani_ids = X_train.loc[X_train['user_id']==sample_user_id, 'anime_id']
not_watched_ids = anime.loc[~anime['anime_id'].isin(watched_ani_ids), 'anime_id']
print('# of animes that not watched yet ::',len(not_watched_ids))

# of animes that not watched yet :: 12281


In [68]:
rating = pd.read_csv(data_path+'rating.csv')

In [74]:
watched_top5 = pd.merge(rating,X_train.loc[X_train['user_id']==sample_user_id,['user_id','anime_id']], on = ['user_id','anime_id'], how='inner').sort_values(by='rating',ascending=False).head(5)
pd.merge(anime[['anime_id','name','genre','type','episodes']], watched_top5, on='anime_id',how='inner')[['name','genre','type','episodes','rating']].sort_values(by='rating',ascending=False)

,name,genre,type,episodes,rating
3,Kaichou wa Maid-sama!,"Comedy, Romance, School, Shoujo",TV,26,10
0,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,9
2,Angel Beats!,"Action, Comedy, Drama, School, Supernatural",TV,13,9
1,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26,7
4,1+2=Paradise,"Comedy, Ecchi, Harem, Romance, Seinen",OVA,2,7


In [77]:
top5_recommend_list = recommend_topk_items(model, sample_user_id, not_watched_ids, user_feature_df, item_feature_df, 5)

In [79]:
pd.merge(anime, pd.DataFrame(top5_recommend_list,columns = ['anime_id','pred_score']), on='anime_id',how='inner')[['name','genre','type','episodes','pred_score']].sort_values(by='pred_score',ascending=False)

,name,genre,type,episodes,pred_score
1,Pokemon XY&amp;Z: Subete no Nazo wo Tokiakase!,"Action, Adventure, Fantasy, Kids",Special,1,1.09
4,Backkom,"Comedy, Kids",TV,52,1.09
0,Line Offline: Salaryman,Comedy,TV,114,1.09
2,Monkey Turn,"Drama, Sports",TV,25,1.09
3,Takamiya Nasuno Desu!: Teekyuu Spin-off Specials,"Comedy, Shounen",Special,2,1.09


In [80]:
print('user_Id : {}'.format(sample_user_id))
print('<<<<< Watched list Top5 >>>>>')
watched_top5 = pd.merge(rating,X_train.loc[X_train['user_id']==sample_user_id,['user_id','anime_id']], on = ['user_id','anime_id'], how='inner').sort_values(by='rating',ascending=False).head(5)
watched_top5_info = pd.merge(anime[['anime_id','name','genre','type','episodes']], watched_top5, on='anime_id',how='inner')[['name','genre','type','episodes','rating']].sort_values(by='rating',ascending=False)
display(watched_top5_info)
print('<<<<< Recommend list >>>>>')
display(pd.merge(anime, pd.DataFrame(top5_recommend_list,columns = ['anime_id','pred_score']), on='anime_id',how='inner')[['name','genre','type','episodes','pred_score']].sort_values(by='pred_score',ascending=False))

user_Id : 20964
<<<<< Watched list Top5 >>>>>


,name,genre,type,episodes,rating
3,Kaichou wa Maid-sama!,"Comedy, Romance, School, Shoujo",TV,26,10
0,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,37,9
2,Angel Beats!,"Action, Comedy, Drama, School, Supernatural",TV,13,9
1,Samurai Champloo,"Action, Adventure, Comedy, Historical, Samurai...",TV,26,7
4,1+2=Paradise,"Comedy, Ecchi, Harem, Romance, Seinen",OVA,2,7


<<<<< Recommend list >>>>>


,name,genre,type,episodes,pred_score
1,Pokemon XY&amp;Z: Subete no Nazo wo Tokiakase!,"Action, Adventure, Fantasy, Kids",Special,1,1.09
4,Backkom,"Comedy, Kids",TV,52,1.09
0,Line Offline: Salaryman,Comedy,TV,114,1.09
2,Monkey Turn,"Drama, Sports",TV,25,1.09
3,Takamiya Nasuno Desu!: Teekyuu Spin-off Specials,"Comedy, Shounen",Special,2,1.09
